# qgear-lightning

## Usage

### Installation

`pip install qgear-lightning`

## How to use

In [ ]:
!pip install qgear-lightning

/bin/bash: /pscratch/sd/g/gzquse/cudaq/lib/libtinfo.so.6: no version information available (required by /lib64/libreadline.so.7)


In [ ]:
import os
import numpy as np
from qiskit import QuantumCircuit
from qgear_lightning.core import qiskit_to_gateList, counts_cudaq_to_qiskit, circ_kernel
from qgear_lightning.io import write4_data_hdf5, read4_data_hdf5
import cudaq
# ----------------------------
# 1. Create a simple Qiskit circuit
n_qubits = 4
qc = QuantumCircuit(n_qubits)
qc.h(0)
qc.cx(0, 1)
qc.ry(np.pi/4, 2)
qc.u(np.pi/2, np.pi/4, np.pi/8, 3)
qc.measure_all()

qcEL = [qc]  # List of circuits
numShots = 10000
verb = 0
backend = "qpp-cpu"  

# ---------------------------
out_path = "out"
os.makedirs(out_path, exist_ok=True)
circ_name = "toy_example"

# Convert to gate list
outD, md = qiskit_to_gateList(qcEL)
inpF = os.path.join(out_path, circ_name + ".gate_list.h5")
md["short_name"] = circ_name
write4_data_hdf5(outD, inpF, md)
num_qubits=0
# Load back
gateD, MD = read4_data_hdf5(inpF, verb)

cudaq.set_target(backend)
nc = len(gateD['circ_type'])
resL = [0] * nc
stateL = [0] * nc
for i in range(nc):
    num_qubit, num_gate = map(int, gateD['circ_type'][i])
    gate_type = list(map(int, gateD['gate_type'][i].flatten()))
    gate_param = list(map(float, gateD['gate_param'][i].flatten()))
    assert num_gate <= len(gate_param)
    prOn = num_qubit < 6 and i == 0 or verb > 1
    num_qubits = num_qubit
    print(cudaq.draw(circ_kernel, num_qubit, num_gate, gate_type, gate_param))


sh: /pscratch/sd/g/gzquse/cudaq/lib/libtinfo.so.6: no version information available (required by /lib64/libreadline.so.7)
sh: /pscratch/sd/g/gzquse/cudaq/lib/libtinfo.so.6: no version information available (required by /lib64/libreadline.so.7)


In [ ]:
circ_kernel